# Continuous Time SIR SDE using `StochasticDiffEq`

### Packages

In [ ]:
# Data Manipulation
using DataFrames
using SparseArrays

# Statistics
using Distributions
using Random

# Data Visualization
using Plots
using PlotThemes

# Numerical Computation
using DifferentialEquations
using StochasticDiffEq
using DiffEqCallbacks

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1260
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for DiffEqBase [2b5f629d-d688-5b77-993f-72d75c75574e]
│   exception = Required dependency ChainRulesCore [d360d2e6-b24c-11e9-a2a3-2a2ae2dbcce4] failed to load from a cache file.
└ @ Base loading.jl:1041


### Flow

In [ ]:
function Φ!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    @inbounds begin
        du[1] = -β*c*I/N*S
        du[2] = β*c*I/N*S - γ*I
        du[3] = γ*I
    end
    nothing
end;

# Make `g` write the sparse matrix values
function Ψ!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    ifrac = β*c*I/N*S
    rfrac = γ*I
    du[1,1] = -sqrt(ifrac)
    du[2,1] = sqrt(ifrac)
    du[2,2] = -sqrt(rfrac)
    du[3,2] = sqrt(rfrac)
end;


function condition(u,t,integrator) # Event when event_f(u,t) == 0
  u[2]
end


function affect!(integrator)
  integrator.u[2] = 0.0
end

cb = ContinuousCallback(condition,affect!);

### Parameters

In [ ]:
# From discrete to continuous time
δt = 0.1
tmax = 40.0
tspan = (0.0,tmax)
t = 0.0:δt:tmax;

𝒯 = tspan;               # Time (t)
ℬ = [990.0,10.0,0.0];    # Initial condition (S, I, R)
𝒫 = [0.05,10.0,0.25];    # Model parameters (β,c,γ)

### Solution

In [ ]:
# Define a sparse matrix by making a dense matrix and setting some values as not zero
A = zeros(3,2)
A[1,1] = 1
A[2,1] = 1
A[2,2] = 1
A[3,2] = 1
A = SparseArrays.sparse(A)

Random.seed!(1234); # Set a random seed

# Problem Definition
problem = SDEProblem(Φ!,Ψ!, ℬ, 𝒯, 𝒫, noise_rate_prototype=A)  

# Problem Solution
solution = solve(problem,SRA1(),callback=cb);

### Visualization 

In [ ]:
# Convert the output to data frame
data = DataFrame(solution(t)')
data[!,:t] = t;

# Select Theme
theme(:default)

@df data plot(:t,
    [:x1 :x2 :x3],
    title = "SIR SDE",
    label=["S" "I" "R"],
    legend=:top,
    xlabel="Time",
    ylabel="Number")